# Lese web_logs som Canvas_history
Først importerer eg alle nødvendige funksjonar:

In [ ]:
import akvut
import pandas as pd

Så les eg `web_logs` frå det siste døgeret (det tok 5'56.7''):

In [3]:

logger = akvut.lag_logger('timer.log')
data = akvut.les_web_logs(logger)
print(data.info())


2025-02-13 12:13:18,652 - my_logger - INFO - Henta access_token OK.
2025-02-13 12:13:18,668 - my_logger - INFO - payload er {"format": "csv", "since": "2025-02-12T12:13:18.668568Z"}
2025-02-13 12:13:18,670 - my_logger - INFO - Sender (inkrementel) spørjing til https://api-gateway.instructure.com/dap/query/canvas_logs/table/web_logs/data


Har ikkje henta frå Azure


2025-02-13 12:13:19,873 - my_logger - INFO - Sjekker status på jobb 5183192b-3f48-4955-ada6-f4d22da13c1a
2025-02-13 12:13:25,509 - my_logger - INFO - Sjekker status på jobb 5183192b-3f48-4955-ada6-f4d22da13c1a
2025-02-13 12:13:30,970 - my_logger - INFO - Sjekker status på jobb 5183192b-3f48-4955-ada6-f4d22da13c1a
2025-02-13 12:13:36,570 - my_logger - INFO - Sjekker status på jobb 5183192b-3f48-4955-ada6-f4d22da13c1a
2025-02-13 12:13:42,055 - my_logger - INFO - Sjekker status på jobb 5183192b-3f48-4955-ada6-f4d22da13c1a
2025-02-13 12:13:47,555 - my_logger - INFO - Sjekker status på jobb 5183192b-3f48-4955-ada6-f4d22da13c1a
2025-02-13 12:13:53,011 - my_logger - INFO - Sjekker status på jobb 5183192b-3f48-4955-ada6-f4d22da13c1a
2025-02-13 12:13:58,508 - my_logger - INFO - Sjekker status på jobb 5183192b-3f48-4955-ada6-f4d22da13c1a
2025-02-13 12:14:03,952 - my_logger - INFO - Sjekker status på jobb 5183192b-3f48-4955-ada6-f4d22da13c1a
2025-02-13 12:14:09,407 - my_logger - INFO - Sjekker st

Feil ved opplasting av sist oppdatert: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect); [IM002] [Microsoft][ODBC Driver Manager] Invalid connection string attribute (0)')
<class 'pandas.core.frame.DataFrame'>
Index: 2193350 entries, 0 to 1703527
Data columns (total 25 columns):
 #   Column                              Dtype 
---  ------                              ----- 
 0   key.id                              object
 1   value.timestamp                     object
 2   value.user_id                       object
 3   value.real_user_id                  object
 4   value.course_id                     object
 5   value.quiz_id                       object
 6   value.discussion_id                 object
 7   value.conversation_id               object
 8   value.assignment_id                 object
 9   value.url                           object
 10  value.http_method                   object
 11  value.

Når eg les `web_logs` lagrer eg rådata i ei fil (`ny_history_alle.csv`). Men eg kan no sende desse vidare til nye rutiner der eg henter ut andre typer data. 

Litt usikker på om det er rett utval eg gjer her.

In [4]:
akvut.ny_history(data, logger)

2025-02-13 12:20:02,531 - my_logger - INFO - Skriv 2193350 linjer i ny_history_alle.csv (som test)
2025-02-13 12:22:26,232 - my_logger - INFO - Antall linjer før reduksjon: 2193350
2025-02-13 12:22:26,458 - my_logger - INFO - Fjerner linjer med api
2025-02-13 12:22:27,994 - my_logger - INFO - Fjerner linjer med retrieve
2025-02-13 12:22:34,744 - my_logger - INFO - Antall linjer etter reduksjon: 340777


## Kven opptrer som kven
Eg prøver å lage statistikk over kven som opptrer som kven. Her har eg `user_id` på dei mest vanlege superbrukarane, og så kan eg lage ei liste over 

- kven dei har opptrådd som
- kor mange ganger dei har opptredd

NB! Det er mange av desse som er "Studentvising", slik at det er mange "user_id" som ikkje finst.


In [5]:
alias = data[~data['value.real_user_id'].isnull()]
adm_namn = {'2916': 'Aasmund', '12353': 'Johan', '3861': 'Casper', '3990': 'Dag Ove', '26737': 'Hege', '97393': 'Henrik', '35965': 'Jan Erik', '14186': 'Andreas Senneset Hove', '6': 'Ivar', '1703': 'Terje'}
adm_liste = []
for adm in ["1703", "2916", "12353", "97393", "3861", "14186", "35965", "98283", "5036", "6", "86785", "26737", "3990"]:
    adm_liste.append({'user': adm, 'count': alias[alias['value.real_user_id'] == adm]['value.user_id'].value_counts()})
for a in adm_liste:
    if len(a['count']) > 0:
        print(f"Administrator: {a['user']} ({adm_namn[a['user']]})")
        for index, item in a['count'].items():
            print(f"{index}: {item}")


Administrator: 2916 (Aasmund)
106420: 148
110226: 27
109587: 12
91290: 3
69294: 2
106055: 2
106968: 2
105587: 2
64804: 1
101453: 1


## Data (til history)
Så til det eg eigentleg skulle gjere: hente ut data til U4-prosjektet. Det er fleire krav:
1. Feltet `user_id` må innehalde noko
2. Feltet `value.http_status` skal være `200`
3. Feltet `value.url` skal ikkje starte på "/api/"
4. Feltet `value.url` skal ikkje innehalde "preview"

4. I feltet `value.web_application_action` skal eg ta vekk alle som er:
 - blanke
 - "content_tag_assignment_data"
 - "progressions"
 - "backup"

In [6]:
temp = data[(data["value.user_id"].notnull()) & (data["value.http_status"].str.startswith("2")) & (~data["value.url"].str.startswith("/api/")) & (~data["value.url"].str.contains("preview"))]
utval = temp[(~temp['value.web_application_action'].isin(['content_tag_assignment_data', 'progressions', 'backup']))]
utval.info()

Eg vil sjå litt nærare på `timestamp`:

In [9]:
utval['value.timestamp']

0          2025-02-12T14:00:02.995Z
9          2025-02-12T14:00:07.027Z
15         2025-02-12T14:00:08.272Z
20         2025-02-12T14:00:08.821Z
40         2025-02-12T14:00:13.398Z
                     ...           
1703487    2025-02-13T02:04:09.374Z
1703492    2025-02-13T02:04:13.621Z
1703493    2025-02-13T02:04:15.684Z
1703517    2025-02-13T02:04:39.371Z
1703520    2025-02-13T02:04:49.946Z
Name: value.timestamp, Length: 215908, dtype: object

In [7]:
utval['value.web_application_action'].value_counts()

value.web_application_action
session_token                60038
show                         41502
user_dashboard               27232
index                        17267
web_app_manifest             17071
                             ...  
final_grade_overrides            1
mobile_login_qr_code             1
reorder_assignments              1
account_calendar_settings        1
confirm                          1
Name: count, Length: 83, dtype: int64

Det er litt usikkert kva vi skal bruke det til, men eg kan sende data til Håvard og be han plukke ut. 

In [10]:
utval.to_csv("history_alle.csv")

Eg kan kanskje sende berre dei som er interessante (håper eg treff på utvalet):

In [12]:
redusert_utval = utval[['value.timestamp', 'value.user_id', 'value.course_id', 'value.url', 'value.web_application_controller', 'value.web_application_action', 'value.web_application_context_type']]


In [13]:
redusert_utval['dato'] = redusert_utval['value.timestamp'].apply(lambda x: x[0:10])

C:\Users\AKV\AppData\Local\Temp\ipykernel_15516\2458858629.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  redusert_utval['dato'] = redusert_utval['value.timestamp'].apply(lambda x: x[0:10])


In [14]:
redusert_utval.to_csv("history_redusert_utval.csv")

Her er ei linje for å hente data inn att frå fil (i tilfelle eg har mista data i løpet av dagen):

In [ ]:
redusert_utval = pd.read_csv("history_250210_redusert_utval.csv")

Eg kan lage ei liste over kor mange aktive hendingar det er per student:

In [15]:
redusert_utval['value.user_id'].value_counts()

value.user_id
102082    394
105833    259
52137     255
104370    250
92822     239
         ... 
109735      1
109981      1
25259       1
1617        1
19466       1
Name: count, Length: 10676, dtype: int64

In [ ]:
redusert_utval

## Leit etter duplikat
Det neste vert å leite etter duplikat. Det vil sei: rader som har lik verdi i fleire celler. Då må eg eigentleg lage ei ny dataramme; eg går gjennom den første og finn like rader, og så legg eg denne raden (og antalet) i ei ny ramme. Vi prøver med noko enkelt:

`user_id`, `course_id`, `web_application_controller`, `web_application_action` og `dato` er like.


In [16]:
temp_duplikat = redusert_utval[["value.user_id", "value.course_id", "value.web_application_controller", "value.web_application_action", "value.web_application_context_type", "dato"]].value_counts()
duplikat = temp_duplikat.to_frame('frekvens').reset_index()

Og så lagrer eg til fil før eg ser på detaljane:

In [31]:
duplikat.to_csv('duplikat.csv')

Først er eg på dei fem mest brukte linjene:

In [17]:
duplikat.head()

,value.user_id,value.course_id,value.web_application_controller,value.web_application_action,value.web_application_context_type,dato,frekvens
0,105833,30023,external_tools,retrieve,Course,2025-02-12,216
1,103192,29164,external_tools,retrieve,Course,2025-02-12,112
2,73365,29869,external_tools,retrieve,Course,2025-02-12,105
3,98878,30023,external_tools,retrieve,Course,2025-02-12,88
4,83680,30297,external_tools,retrieve,Course,2025-02-13,80


Her har altså student 105833 gjort same kommando 216 ganger! Dette er kall til "external_tools", så eg må sjekke eksakt *kva* verktøy dette er. (Det var Panopto)

In [24]:
student_71923 = data[(data['value.user_id'] == "71923") & (data['value.web_application_action'] == "image_thumbnail")]
student_71923.to_csv('student_71923.csv', index=False)

Så tar eg ut ei liste over kor mange ganger kva duplikat-rad opptrer; det viser seg jo at dei aller, aller fleste radene er unike:

In [20]:
duplikat['frekvens'].value_counts().sort_index()

frekvens
1      19772
2       6656
3       2812
4       1426
5        742
6        482
7        244
8        203
9        121
10       102
11        88
12        58
13        32
14        33
15        23
16        22
17        12
18        17
19        14
20        18
21        12
22        10
23         3
24        10
25         4
26         5
27         1
28         8
29         3
30         7
31         5
32         4
33         3
34         3
35         2
37         2
40         4
41         2
42         2
43         2
48         2
52         1
53         1
54         1
55         3
59         1
62         1
72         1
80         1
88         1
105        1
112        1
216        1
Name: count, dtype: int64

## Antal unike wiki_pages besøkt
Vi kan også sjå på kor mange ganger kvar enkelt wiki_side er besøkt:

In [21]:
temp99 = redusert_utval[redusert_utval["value.web_application_controller"]=="wiki_pages"]
url = temp99[["value.user_id", "value.course_id", "value.url"]].value_counts()
url_ramme = url.to_frame('frekvens').reset_index()

In [22]:
url_ramme.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4179 entries, 0 to 4178
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   value.user_id    4179 non-null   object
 1   value.course_id  4179 non-null   object
 2   value.url        4179 non-null   object
 3   frekvens         4179 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 130.7+ KB


In [31]:
url_ramme

,value.user_id,value.course_id,value.url,frekvens
0,52137,30810,/courses/30810/pages/pensum-og-forberedelsesma...,25
1,52137,30810,/courses/30810/pages/pensum-og-forberedelsesma...,23
2,1703,86,/courses/86/pages/alt-1-guide,22
3,52137,30810,/courses/30810/pages/pensum-og-forberedelsesma...,16
4,107515,30017,/courses/30017/pages/uke-7-kjernefysikk-strali...,14
...,...,...,...,...
4174,40208,29744,/courses/29744/pages/framside,1
4175,40208,29744,/courses/29744/pages/framside/edit,1
4176,40229,29317,/courses/29317/pages/obligatoriske-laeringsakt...,1
4177,40382,25541,/courses/25541/pages/sql,1


Eg vil no beholde berre *ei* av linjene dersom det er fleire besøk til ei side (sjølvsagt per student), men lage ei ekstra kolonne med antalet (henta frå lista over).

In [32]:
redusert_utval.info()

<class 'pandas.core.frame.DataFrame'>
Index: 215908 entries, 0 to 1703520
Data columns (total 8 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   value.timestamp                     215908 non-null  object
 1   value.user_id                       215908 non-null  object
 2   value.course_id                     83053 non-null   object
 3   value.url                           215908 non-null  object
 4   value.web_application_controller    215908 non-null  object
 5   value.web_application_action        215908 non-null  object
 6   value.web_application_context_type  110229 non-null  object
 7   dato                                215908 non-null  object
dtypes: object(8)
memory usage: 14.8+ MB


In [42]:
redusert_utval = redusert_utval.reset_index(drop=True)
temp999 = redusert_utval[redusert_utval["value.web_application_controller"]=="wiki_pages"]
redusert_utval['sideklikk'] = temp999.groupby(['value.user_id', 'value.url'])['value.user_id'].transform('count')

In [44]:
redusert_utval[redusert_utval['value.web_application_controller']=="wiki_pages"]

,value.timestamp,value.user_id,value.course_id,value.url,value.web_application_controller,value.web_application_action,value.web_application_context_type,dato,sideklikk
29,2025-02-12T14:00:55.428Z,108814,25064,/courses/25064/pages/kildehenvisning-eksempel?...,wiki_pages,show,Course,2025-02-12,1.0
44,2025-02-12T14:01:12.366Z,90945,29659,/courses/29659/pages/fordypningsoppgavefiler?m...,wiki_pages,show,Course,2025-02-12,1.0
52,2025-02-12T14:01:18.330Z,106607,29295,/courses/29295/pages/artikkelseminar-3-og-4-kv...,wiki_pages,show,Course,2025-02-12,1.0
60,2025-02-12T14:01:23.791Z,108814,25064,/courses/25064/pages/eksempel-pa-eksamensoppga...,wiki_pages,show,Course,2025-02-12,1.0
65,2025-02-12T14:01:32.313Z,101596,28886,/courses/28886/pages/obligatorisk-gruppearbeid...,wiki_pages,show,Course,2025-02-12,3.0
...,...,...,...,...,...,...,...,...,...
215181,2025-02-13T01:21:34.384Z,67496,29464,/courses/29464/pages/19-dot-02-dot-2025-utvikl...,wiki_pages,show,Course,2025-02-13,2.0
215459,2025-02-13T01:32:58.603Z,106405,29313,/courses/29313/pages/deskriptiv-statistikk-2?m...,wiki_pages,show,Course,2025-02-13,1.0
215793,2025-02-13T01:55:41.058Z,83608,28959,/courses/28959/pages/oppgaver-fra-workshop-for...,wiki_pages,show,Course,2025-02-13,1.0
215845,2025-02-13T01:59:25.480Z,83608,28959,/courses/28959/pages/farmakologi-oppgaver-2?mo...,wiki_pages,show,Course,2025-02-13,1.0


In [46]:
redusert_utval['sideklikk'].value_counts().sort_index()

sideklikk
1.0     2987
2.0     1562
3.0      777
4.0      376
5.0      210
6.0       84
7.0       63
8.0       16
9.0       27
10.0      10
11.0      22
13.0      13
14.0      14
16.0      16
22.0      22
23.0      23
25.0      25
Name: count, dtype: int64